In [1]:
!pip install git+https://github.com/mindspore-lab/mindnlp.git@master

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple/
  Cloning https://github.com/mindspore-lab/mindnlp.git (to revision master) to /tmp/pip-req-build-bfsbkd4d
  Running command git clone --filter=blob:none --quiet https://github.com/mindspore-lab/mindnlp.git /tmp/pip-req-build-bfsbkd4d
  Resolved https://github.com/mindspore-lab/mindnlp.git to commit 6c357b83653eefcb26dcc3e956d532f2bb700ff5
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.0/782.0 kB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 3.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.3 MB/s eta 0:00:00
  Created wheel for mindnlp: filename=mindnlp-0.4.1-py3-none-any.whl size=8744119 sha256=32721db43d56c5a6f78f3a7910414d97580db721b169a41

In [2]:
pip show mindnlp

Name: mindnlp
Version: 0.4.1
Summary: An open source natural language processing research tool box. Git version: [sha1]:6c357b83, [branch]: (HEAD -> master, origin/master, origin/HEAD)
Home-page: https://github.com/mindlab-ai/mindnlp/tree/master/
Author: MindSpore Team
Author-email: 
License: Apache 2.0
Location: /home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages
Requires: addict, datasets, evaluate, mindspore, ml-dtypes, pillow, pyctcdecode, pytest, regex, requests, safetensors, sentencepiece, tokenizers, tqdm
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
from tqdm import tqdm
import mindspore
from mindnlp.core.optim import AdamW
from mindnlp.dataset import load_dataset
import mindnlp.peft as peft

import mindnlp.evaluate as evaluate
from mindnlp.dataset import load_dataset
from mindnlp.common.optimization import get_linear_schedule_with_warmup
from mindnlp.transformers import AutoModelForSequenceClassification, AutoTokenizer

/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [4]:
from mindspore import context

context.set_context(device_target="Ascend")

In [5]:
batch_size = 8
model_name_or_path = "AI-ModelScope/roberta-large"
task = "mrpc"
peft_type = peft.PeftType.IA3
num_epochs = 6

peft_config = peft.IA3Config(task_type="SEQ_CLS", inference_mode=False)
lr = 1e-3

In [6]:
if any(k in model_name_or_path for k in ("gpt", "opt", "bloom")):
    padding_side = "left"
else:
    padding_side = "right"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side=padding_side, mirror='modelscope')
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

  0%|          | 0.00/482 [00:00<?, ?B/s]

  0%|          | 0.00/878k [00:00<?, ?B/s]

  0%|          | 0.00/446k [00:00<?, ?B/s]

  0%|          | 0.00/1.29M [00:00<?, ?B/s]

/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted, and will be then set to `False` by default. 
  warnings.warn(


In [7]:
datasets = load_dataset("glue", task)
print(next(datasets['train'].create_dict_iterator()))

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

{'sentence1': Tensor(shape=[], dtype=String, value= 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .'), 'sentence2': Tensor(shape=[], dtype=String, value= 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'), 'label': Tensor(shape=[], dtype=Int64, value= 1), 'idx': Tensor(shape=[], dtype=Int64, value= 0)}


In [8]:
from mindnlp.dataset import BaseMapFunction

class MapFunc(BaseMapFunction):
    def __call__(self, sentence1, sentence2, label, idx):
        outputs = tokenizer(sentence1, sentence2, truncation=True, max_length=None)
        return outputs['input_ids'], outputs['attention_mask'], label


def get_dataset(dataset, tokenizer):
    input_colums=['sentence1', 'sentence2', 'label', 'idx']
    output_columns=['input_ids', 'attention_mask', 'labels']
    dataset = dataset.map(MapFunc(input_colums, output_columns),
                          input_colums, output_columns)
    dataset = dataset.padded_batch(batch_size, pad_info={'input_ids': (None, tokenizer.pad_token_id),
                                                         'attention_mask': (None, 0)})
    return dataset

train_dataset = get_dataset(datasets['train'], tokenizer)
eval_dataset = get_dataset(datasets['validation'], tokenizer)
test_dataset = get_dataset(datasets['test'], tokenizer)
print(next(train_dataset.create_dict_iterator()))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': Tensor(shape=[8, 67], dtype=Int64, value=
[[    0, 10127,  1001 ...     1,     1,     1],
 [    0,   975, 26802 ...     1,     1,     1],
 [    0,  1213,    56 ...     1,     1,     1],
 ...
 [    0,  9064, 32497 ...     1,     1,     1],
 [    0,   133,  4417 ...     1,     1,     1],
 [    0,   133, 19888 ...     1,     1,     1]]), 'attention_mask': Tensor(shape=[8, 67], dtype=Int64, value=
[[1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 ...
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0]]), 'labels': Tensor(shape=[8], dtype=Int64, value= [1, 0, 1, 0, 1, 1, 0, 1])}


In [9]:
metric = evaluate.load("glue", task)

In [10]:
model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, mirror='modelscope')
model = peft.get_peft_model(model, peft_config)
model.print_trainable_parameters()

  0%|          | 0.00/1.32G [00:00<?, ?B/s]

[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at AI-ModelScope/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,223,682 || all params: 356,585,476 || trainable%: 0.34316652874555104


In [11]:
optimizer = AdamW(params=model.trainable_params(), lr=lr)
# Instantiate scheduler
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0.06 * (len(train_dataset) * num_epochs),
    num_training_steps=(len(train_dataset) * num_epochs),
)

In [12]:
from mindnlp.core import value_and_grad

best_model = None
best_metric = None

def forward_fn(**batch):
    outputs = model(**batch)
    loss = outputs.loss
    return loss

grad_fn = value_and_grad(forward_fn, tuple(model.trainable_params()))

for epoch in range(num_epochs):
    model.set_train()
    train_total_size = train_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(train_dataset.create_dict_iterator(), total=train_total_size)):
        optimizer.zero_grad()
        loss = grad_fn(**batch)
        optimizer.step()
        lr_scheduler.step()

    model.set_train(False)
    eval_total_size = eval_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(eval_dataset.create_dict_iterator(), total=eval_total_size)):
        outputs = model(**batch)
        predictions = outputs.logits.argmax(axis=-1)
        predictions, references = predictions, batch["labels"]
        metric.add_batch(
            predictions=predictions,
            references=references,
        )

    eval_metric = metric.compute()
    print(f"epoch {epoch}:", eval_metric)

  0%|          | 0/459 [00:00<?, ?it/s]

  0%|          | 1/459 [00:18<2:21:18, 18.51s/it]

  2%|▏         | 1/51 [00:00<00:37,  1.34it/s]

100%|██████████| 51/51 [00:05<00:00,  9.71it/s]


epoch 0: {'accuracy': 0.6838235294117647, 'f1': 0.8122270742358079}


...

epoch 3: {'accuracy': 0.8676470588235294, 'f1': 0.9021739130434783}


100%|██████████| 51/51 [00:04<00:00, 10.52it/s]


epoch 4: {'accuracy': 0.8799019607843137, 'f1': 0.911070780399274}


100%|██████████| 51/51 [00:04<00:00, 10.27it/s]


epoch 5: {'accuracy': 0.8602941176470589, 'f1': 0.9001751313485113}


In [15]:
# saving model
peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"
model.save_pretrained(peft_model_id)

In [16]:
ckpt = f"{peft_model_id}/adapter_model.ckpt"
!du -h $ckpt

4.7M	AI-ModelScope/roberta-large_IA3_SEQ_CLS/adapter_model.ckpt


In [17]:
from mindnlp.peft import PeftModel, IA3Config

config = IA3Config.from_pretrained(peft_model_id)
model = AutoModelForSequenceClassification.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at AI-ModelScope/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from mindspore import Tensor

def predict(sentence1, sentence2):
    inputs = tokenizer(sentence1, sentence2, truncation=True, padding=True, max_length=128, return_tensors="ms")
    outputs = model(Tensor(inputs['input_ids']), Tensor(inputs['attention_mask']))
    logits = outputs.logits
    prediction = logits.argmax(axis=-1).asnumpy()[0]

    if prediction == 1:
        return "同义"
    else:
        return "同义"
    
sentence1 = 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .'
sentence2 = 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'

print(predict(sentence1, sentence2))

同义


In [24]:
import gradio as gr

iface = gr.Interface(fn=predict, 
                     inputs=["text", "text"], 
                     outputs="text", 
                     title="文本分类模型",
                     description="请输入两个句子进行分类，判断它们是否同义")

iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7862


/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(



Could not create share link. Missing file: /home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/gradio/frpc_linux_aarch64_v0.2. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_linux_aarch64
2. Rename the downloaded file to: frpc_linux_aarch64_v0.2
3. Move the file to this location: /home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/gradio


In [15]:
pip show gradio

Name: gradio
Version: 4.44.0
Summary: Python library for easily interacting with trained machine learning models
Home-page: https://github.com/gradio-app/gradio
Author: 
Author-email: Abubakar Abid <gradio-team@huggingface.co>, Ali Abid <gradio-team@huggingface.co>, Ali Abdalla <gradio-team@huggingface.co>, Dawood Khan <gradio-team@huggingface.co>, Ahsen Khaliq <gradio-team@huggingface.co>, Pete Allen <gradio-team@huggingface.co>, Ömer Faruk Özdemir <gradio-team@huggingface.co>, Freddy A Boulton <gradio-team@huggingface.co>, Hannah Blair <gradio-team@huggingface.co>
License: 
Location: /home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages
Requires: aiofiles, anyio, fastapi, ffmpy, gradio-client, httpx, huggingface-hub, importlib-resources, jinja2, markupsafe, matplotlib, numpy, orjson, packaging, pandas, pillow, pydantic, pydub, python-multipart, pyyaml, ruff, semantic-version, tomlkit, typer, typing-extensions, urllib3, uvicorn
Required-by: 
Note: you may need to restart 

In [18]:
import os

root_dir = '/'

for dirpath, dirnames, filenames in os.walk(root_dir):
    if "frpc_windows_amd64_v0.2" in filenames:
        print(f"Found at: {os.path.join(dirpath, 'frpc_windows_amd64_v0.2')}")


Found at: /home/mindspore/work/frpc_windows_amd64_v0.2


In [23]:
!mv /home/mindspore/work/frpc_linux_amd64 /home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/gradio

In [22]:
pip install https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_linux_aarch64 -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
ERROR: Could not install packages due to an OSError: HTTPSConnectionPool(host='cdn-media.huggingface.co', port=443): Max retries exceeded with url: /frpc-gradio-0.2/frpc_linux_aarch64 (Caused by ConnectTimeoutError(<pip._vendor.urllib3.connection.HTTPSConnection object at 0xffffa2c5fb80>, 'Connection to cdn-media.huggingface.co timed out. (connect timeout=15)'))

Note: you may need to restart the kernel to use updated packages.
